In [6]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from scipy.integrate import odeint
import pprint
from ipywidgets import interact, interactive, fixed, interact_manual, jslink
import ipywidgets as widgets
import ipyvolume as ipv
import ipyvolume.pylab as p3
import traitlets
import time, sys
from IPython.core.display import clear_output

In [249]:
# define the atoms in the chain (this test will only use a 3 atom chain)
r1 = (0, 0, 1)
r2 = (2, 1, 1)
r3 = (3, 3, 1)

floaters = np.hstack((r1, r2, r3))

m = 3             # total atoms in the chain

In [250]:
# define parameters
beta = 1

In [251]:
# define time interval
t = np.linspace(0,6,10)

In [256]:
# bending force DE system
def bending_force(floaters, t):
    
    current_bending_force = np.zeros((m,3))
    total_bending_force = np.zeros(m)
    
    # loop for computing bending force
    for i in range(1,m-1):          # loop through each atom not on the end of the chain and compute the bending force on it and its neighbors
        
        print(i)
        # define the pieces of the bending energy equation
        sx_n = floaters[i*3] - floaters[(i-1)*3]
        sx_n1 = floaters[(i+1)*3] - floaters[i*3]

        sy_n = floaters[i*3 + 1] - floaters[(i-1)*3 + 1]
        sy_n1 = floaters[(i+1)*3 + 1] - floaters[i*3 + 1]

        sz_n = floaters[i*3 + 2] - floaters[(i-1)*3 + 2]
        sz_n1 = floaters[(i+1)*3 + 2] - floaters[i*3 + 2]

        A = norm((sx_n, sy_n, sz_n))*norm((sx_n1, sy_n1, sz_n1))

        B = np.dot((sx_n, sy_n, sz_n),(sx_n1, sy_n1, sz_n1))

        partial_common = .5/A
        
        # partials of A wrt x for atom n, n-1 and n+1, respectively
        Axn = partial_common*(2*(sx_n)*(sx_n1**2) - 2*(sx_n1)*(sx_n**2) + 2*(sx_n)*(sy_n1**2) + 2*(sx_n)*(sz_n1**2) - 2*(sx_n1)*(sy_n**2) - 2*(sx_n1)*(sz_n**2))
        Axn0 = partial_common*(-2*(sx_n)*(sx_n1**2) - 2*(sx_n)*(sy_n1**2) - 2*(sx_n)*(sz_n1**2))
        Axn1 = partial_common*(2*(sx_n**2)*(sx_n1) + 2*(sy_n**2)*(sx_n1) + 2*(sz_n**2)*(sx_n1))
        
        # partials of A wrt y for atom n, n-1 and n+1, respectively
        Ayn = partial_common*(-2*(sx_n**2)*(sy_n1) + 2*(sy_n)*(sx_n1**2) + 2*(sy_n)*(sy_n1**2) - 2*(sy_n**2)*(sy_n1) + 2*(sy_n)*(sz_n1**2) - 2*(sz_n**2)*(sy_n1))
        Ayn0 = partial_common*(-2*(sy_n)*(sx_n1**2) - 2*(sy_n)*(sy_n1**2) - 2*(sy_n)*(sz_n1**2))
        Ayn1 = partial_common*(2*(sy_n**2)*(sy_n1) + 2*(sz_n**2)*(sy_n1) + 2*(sx_n**2)*(sy_n1))
        
        # partials of A wrt z for atom n, n-1 and n+1, respectively
        Azn = partial_common*(-2*(sx_n**2)*(sz_n1) - 2*(sy_n**2)*(sz_n1) - 2*(sz_n**2)*(sz_n1) + 2*(sz_n)*(sz_n1**2) + 2*(sz_n)*(sx_n1**2) + 2*(sz_n)*(sy_n1**2))
        Azn0 = partial_common*(-2*(sz_n)*(sx_n1**2) -2*(sz_n)*(sy_n1**2) - 2*(sz_n)*(sz_n1**2))
        Azn1 = partial_common*(2*(sz_n1)*(sx_n**2) + 2*(sz_n1)*(sy_n**2) + 2*(sz_n1)*(sz_n**2))
        
        # partials of B wrt x for atom n, n-1 and n+1, respectively
        Bxn = floaters[(i+1)*3] - 2*floaters[i*3] + floaters[(i-1)*3]
        Bxn0 = -floaters[(i+1)*3] + floaters[i*3]
        Bxn1 = floaters[i*3] - floaters[(i-1)*3]

        # partials of B wrt y for atom n, n-1 and n+1, respectively
        Byn = floaters[(i+1)*3 + 1] - 2*floaters[i*3 + 1] + floaters[(i-1)*3 + 1]
        Byn0 = -floaters[(i+1)*3 + 1] + floaters[i*3 + 1]
        Byn1 = floaters[i*3 + 1] - floaters[(i-1)*3 + 1]
        
        # partials of B wrt z for atom n, n-1 and n+1, respectively
        Bzn = floaters[(i+1)*3 + 2] - 2*floaters[i*3 + 2] + floaters[(i-1)*3 + 2]
        Bzn0 = -floaters[(i+1)*3 + 2] + floaters[i*3 + 2]
        Bzn1 = floaters[i*3 + 2] - floaters[(i-1)*3 + 2]
        
        try:
            print("try")
            Eb_common = 4*beta/((A+B)**2)                   # compute common part of bending energy gradient
            if (np.isnan(Eb_common)  or np.isinf(Eb_common) or np.isnan(A)):
                raise ValueError('Divide by zero')
                raise Exception
            current_bending_force[i][0] = Eb_common*(B*Axn - A*Bxn)
            current_bending_force[i][1] = Eb_common*(B*Ayn - A*Byn)
            current_bending_force[i][2] = Eb_common*(B*Azn - A*Bzn)
            
            current_bending_force[i-1][0] = Eb_common*(B*Axn0 - A*Bxn0)
            current_bending_force[i-1][1] = Eb_common*(B*Ayn0 - A*Byn0)
            current_bending_force[i-1][2] = Eb_common*(B*Azn0 - A*Bzn0)
            
            current_bending_force[i+1][0] = Eb_common*(B*Axn1 - A*Bxn1)
            current_bending_force[i+1][1] = Eb_common*(B*Ayn1 - A*Byn1)
            current_bending_force[i+1][2] = Eb_common*(B*Azn1 - A*Bzn1)
            
        except Exception as error:
            print("except")
            print("Caught exception: ", error)
            continue
            

    total_bending_force = np.hstack(current_bending_force)
    print(total_bending_force)
    return total_bending_force

In [257]:
%%time

# solve the system of ODEs to compute changing positions as a result of the bending force acting on the chain
gFlow = odeint(bending_force, floaters, t, atol=1.4e-10)

1
try
[-0.14814815  0.2962963  -0.          0.44444444 -0.44444444  0.         -0.2962963
  0.14814815  0.        ]
1
try
[-0.14814857  0.29629833 -0.          0.44444691 -0.44444691  0.
 -0.29629833  0.14814857  0.        ]
1
try
[-0.14814857  0.29629833 -0.          0.44444691 -0.44444691  0.
 -0.29629833  0.14814857  0.        ]
1
try
[-0.148149    0.29630037 -0.          0.44444937 -0.44444937  0.
 -0.29630037  0.148149    0.        ]
1
try
[-0.148149    0.29630037 -0.          0.44444937 -0.44444937  0.
 -0.29630037  0.148149    0.        ]
1
try
[-0.14823698  0.29672323 -0.          0.44496022 -0.44496022  0.
 -0.29672323  0.14823698  0.        ]
1
try
[-0.14823698  0.29672323 -0.          0.44496022 -0.44496022  0.
 -0.29672323  0.14823698  0.        ]
1
try
[-0.14832491  0.29714686 -0.          0.44547177 -0.44547177  0.
 -0.29714686  0.14832491  0.        ]
1
try
[-0.14832491  0.29714686 -0.          0.44547177 -0.44547177  0.
 -0.29714686  0.14832491  0.        ]
1
try
[-0.14

 -11.42509976  -5.1404977   -0.        ]
1
try
[  5.56128474  12.10612308   0.           6.54483834  -6.54483834  -0.
 -12.10612308  -5.56128474  -0.        ]
1
try
[  5.56128575  12.10612472   0.           6.54483897  -6.54483897  -0.
 -12.10612472  -5.56128575  -0.        ]
1
try
[  6.04891208  12.88660146   0.           6.83768938  -6.83768938  -0.
 -12.88660146  -6.04891208  -0.        ]
1
try
[  6.04891407  12.88660467   0.           6.83769061  -6.83769061  -0.
 -12.88660467  -6.04891407  -0.        ]
1
try
[  5.94881537  12.72710588   0.           6.77829051  -6.77829051  -0.
 -12.72710588  -5.94881537  -0.        ]
1
try
[  5.94881629  12.72710736   0.           6.77829108  -6.77829108  -0.
 -12.72710736  -5.94881629  -0.        ]
1
try
[  6.3887092   13.42545019   0.           7.03674099  -7.03674099  -0.
 -13.42545019  -6.3887092   -0.        ]
1
try
[  6.38870939  13.42545049   0.           7.0367411   -7.0367411   -0.
 -13.42545049  -6.38870939  -0.        ]
1
try
[  6.8925

1
try
[ 2365.74895529  2632.42464076     0.           266.67568547  -266.67568547
    -0.         -2632.42464076 -2365.74895529    -0.        ]
1
try
[ 2597.45909104  2880.92964889     0.           283.47055784  -283.47055784
    -0.         -2880.92964889 -2597.45909104    -0.        ]
1
try
[ 2597.4605045   2880.93116328     0.           283.47065878  -283.47065878
    -0.         -2880.93116328 -2597.4605045     -0.        ]
1
try
[ 2888.92019967  3192.81177854     0.           303.89157886  -303.89157886
    -0.         -3192.81177854 -2888.92019967    -0.        ]
1
try
[ 2888.94463784  3192.8379         0.           303.89326215  -303.89326215
    -0.         -3192.8379     -2888.94463784    -0.        ]
1
try
[ 2825.63064512  3125.1495981      0.           299.51895298  -299.51895298
    -0.         -3125.1495981  -2825.63064512    -0.        ]
1
try
[ 2825.64162505  3125.16134003     0.           299.51971498  -299.51971498
    -0.         -3125.16134003 -2825.64162505    -0.  

1
try
[  1.19996305e+09   1.20159669e+09   0.00000000e+00   1.63363876e+06
  -1.63363876e+06  -0.00000000e+00  -1.20159669e+09  -1.19996305e+09
  -0.00000000e+00]
1
try
[  1.20031349e+09   1.20194745e+09   0.00000000e+00   1.63395673e+06
  -1.63395673e+06  -0.00000000e+00  -1.20194745e+09  -1.20031349e+09
  -0.00000000e+00]
1
try
[  1.52592975e+09   1.52784718e+09   0.00000000e+00   1.91742605e+06
  -1.91742605e+06  -0.00000000e+00  -1.52784718e+09  -1.52592975e+09
  -0.00000000e+00]
1
try
[  1.52812894e+09   1.53004820e+09   0.00000000e+00   1.91926750e+06
  -1.91926750e+06  -0.00000000e+00  -1.53004820e+09  -1.52812894e+09
  -0.00000000e+00]
1
try
[  1.39512008e+09   1.39692632e+09   0.00000000e+00   1.80624022e+06
  -1.80624022e+06  -0.00000000e+00  -1.39692632e+09  -1.39512008e+09
  -0.00000000e+00]
1
try
[  1.39561865e+09   1.39742532e+09   0.00000000e+00   1.80667043e+06
  -1.80667043e+06  -0.00000000e+00  -1.39742532e+09  -1.39561865e+09
  -0.00000000e+00]
1
try
[  1.68527598e+0

/Users/christopher/.anaconda/ipykernel/ipykernel_launcher.py:59: RuntimeWarning: divide by zero encountered in double_scalars



[  7.24354261e+21   7.24354315e+21  -5.46389460e+20   5.21077643e+14
  -5.58297474e+14  -2.56511154e+06  -7.24354313e+21  -7.24354263e+21
   5.46389460e+20]
1
try
[  7.24354269e+21   7.24354338e+21   0.00000000e+00   4.83857811e+14
  -5.21077643e+14  -0.00000000e+00  -7.24354321e+21  -7.24354284e+21
  -0.00000000e+00]
1
try
[  7.94982906e+21   7.94982967e+21   0.00000000e+00   6.12734643e+14
  -5.71885667e+14  -0.00000000e+00  -7.94982963e+21  -7.94982912e+21
  -0.00000000e+00]
1
try
[  7.24354261e+21   7.24354315e+21  -2.56511154e+06   5.21077643e+14
  -5.58297474e+14   3.05040291e+06  -7.24354313e+21  -7.24354263e+21
  -4.15964033e+05]
1
try
[  6.26673457e+21   6.26673507e+21   0.00000000e+00   5.31674956e+14
  -4.69124961e+14  -0.00000000e+00  -6.26673508e+21  -6.26673458e+21
  -0.00000000e+00]
1
try
[  5.61951023e+21   5.61951066e+21   0.00000000e+00   4.79673333e+14
  -5.06321852e+14  -0.00000000e+00  -5.61951064e+21  -5.61951015e+21
  -0.00000000e+00]
1
try
[  7.24354261e+21   7

try
[  2.87846003e+23   2.87846007e+23   0.00000000e+00   0.00000000e+00
  -4.50359963e+15  -0.00000000e+00  -2.87846010e+23  -2.87846003e+23
  -0.00000000e+00]
1
try
[  8.58469757e+23   8.58469766e+23   2.64452499e+23  -1.80143985e+16
   0.00000000e+00  -1.34888817e+10  -8.58469775e+23  -8.58469775e+23
  -2.64452499e+23]
1
try
except
Caught exception:  Divide by zero
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
1
try
[ -7.33825810e+21  -7.33825759e+21   0.00000000e+00   4.83857811e+14
  -5.21077643e+14  -0.00000000e+00   7.33825759e+21   7.33825810e+21
  -0.00000000e+00]
1
try
[ -3.34807639e+20  -3.34807570e+20   0.00000000e+00   6.89994403e+13
  -6.96155067e+13  -0.00000000e+00   3.34807570e+20   3.34807639e+20
  -0.00000000e+00]
1
try
[ -4.19252550e+19  -4.19252376e+19   0.00000000e+00   1.74162600e+13
  -1.74548770e+13  -0.00000000e+00   4.19252376e+19   4.19252550e+19
  -0.00000000e+00]
1
try
[ -5.07012617e+18  -5.07012191e+18   0.00000000e+00   4.26830882e+12
  -4.26599787e+12  -0.00000

/anaconda/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [258]:
print(gFlow)

[[  0.00000000e+00   0.00000000e+00   1.00000000e+00   2.00000000e+00
    1.00000000e+00   1.00000000e+00   3.00000000e+00   3.00000000e+00
    1.00000000e+00]
 [ -9.71291401e-02   4.41357370e-01   1.00000000e+00   2.53848651e+00
    4.61513490e-01   1.00000000e+00   2.55864263e+00   3.09712914e+00
    1.00000000e+00]
 [  7.84747635e-01   2.21524843e+00   1.00000000e+00   3.43050079e+00
   -4.30500793e-01   1.00000000e+00   7.84751572e-01   2.21525237e+00
    1.00000000e+00]
 [  9.03864388e-17   9.03864388e-17   1.00000000e+00   1.00000000e+00
    1.00000000e+00   1.00000000e+00   3.00000000e+00   3.00000000e+00
    1.00000000e+00]
 [  9.03864388e-17   9.03864388e-17   1.00000000e+00   1.00000000e+00
    1.00000000e+00   1.00000000e+00   3.00000000e+00   3.00000000e+00
    1.00000000e+00]
 [  9.03864388e-17   9.03864388e-17   1.00000000e+00   1.00000000e+00
    1.00000000e+00   1.00000000e+00   3.00000000e+00   3.00000000e+00
    1.00000000e+00]
 [  9.03864388e-17   9.03864388e-17   1.

In [259]:
sx_n = gFlow[-1][3] - gFlow[-1][0]
sx_n1 = gFlow[-1][6] - gFlow[-1][3]

sy_n = gFlow[-1][4] - gFlow[-1][1]
sy_n1 = gFlow[-1][7] - gFlow[-1][4]

sz_n = gFlow[-1][5] - gFlow[-1][2]
sz_n1 = gFlow[-1][8] - gFlow[-1][5]

A = norm((sx_n, sy_n, sz_n))*norm((sx_n1, sy_n1, sz_n1))

B = np.dot((sx_n, sy_n, sz_n),(sx_n1, sy_n1, sz_n1))

print(A, B, A-B, A+B, 2*beta*(A-B)/(A+B))

4.0 4.0 0.0 8.0 0.0
